In [14]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim
#gpuの確認
print(torch.cuda.is_available())

True


In [15]:
import torchtext
from torchtext.data.utils import get_tokenizer

toke = get_tokenizer('spacy')
toke('I am N')

['I', 'am', 'N']

In [16]:
import torchtext
from torchtext.data.utils import get_tokenizer

#テキストに処理を行うFieldを定義
#fix_lengthはtokenの数
TEXT = torchtext.data.Field(sequential=True, use_vocab=True, tokenize=get_tokenizer('spacy'),
                            lower=True, include_lengths=True, batch_first=True, fix_length=37)

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

#pandasでcsvを保存するときに、labelをintでキャストしておかないとエラーでるから注意
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='drive/My Drive/dataset/CBET/ekman', train='train.csv', validation='val.csv', 
    test='test.csv', format='csv', fields=[('text', TEXT), ('Label', LABEL)])

In [17]:
#ボキャブラリを作成する
TEXT.build_vocab(train_ds)
#TEXT.build_vocab(train_ds, vectors=japanese_word2vec_vectors) #学習ずみの分散表現を使う場合
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f16c15debf8>, {'<unk>': 0, '<pad>': 1, '#': 2, 'the': 3, '!': 4, 'i': 5, 'to': 6, 'a': 7, '@user': 8, 'of': 9, 'you': 10, 'and': 11, 'is': 12, 'in': 13, '?': 14, 'my': 15, 'for': 16, 'on': 17, 'it': 18, 'me': 19, 'that': 20, 'nt': 21, 'this': 22, 'do': 23, 'with': 24, 'not': 25, 'be': 26, 'so': 27, 'your': 28, 's': 29, 'are': 30, 'have': 31, 'can': 32, 'at': 33, 'just': 34, 'life': 35, 'no': 36, 'what': 37, 'm': 38, 'day': 39, 'like': 40, 'all': 41, 'people': 42, 'love': 43, 'when': 44, 'its': 45, 'but': 46, 'we': 47, 'up': 48, 'out': 49, 'from': 50, 'how': 51, 'get': 52, 'was': 53, 'good': 54, 'fear': 55, 'now': 56, 'one': 57, 'time': 58, 'about': 59, 'by': 60, 'they': 61, 'if': 62, 'will': 63, 'why': 64, 'insta': 65, 'as': 66, 't': 67, 'free': 68, 'new': 69, 'who': 70, 'an': 71, 'today': 72, 'or': 73, 'more': 74, 'he': 75, 'night': 76, 'go': 77, 'smile': 78, 'happy': 79, 'live': 80, 'girl': 81, 'there': 82, 'chat': 83, 'self': 84, 'our':

In [18]:
#データローダーを作成
train_dl = torchtext.data.Iterator(train_ds, batch_size=64, train=True)
val_dl = torchtext.data.Iterator(val_ds, batch_size=64, train=False, sort=False)
test_dl = torchtext.data.Iterator(test_ds, batch_size=64, train=False, sort=False)

#テスト
batch = next(iter(val_dl))
print(len(batch.text[0][0]))
print(batch.Label)

37
tensor([4, 0, 2, 0, 2, 4, 4, 3, 1, 4, 2, 2, 1, 2, 2, 2, 1, 2, 3, 3, 0, 3, 4, 1,
        1, 3, 0, 2, 4, 1, 2, 2, 2, 3, 1, 2, 4, 3, 3, 4, 0, 0, 1, 3, 0, 3, 4, 0,
        0, 1, 2, 2, 1, 1, 4, 3, 2, 3, 3, 3, 2, 4, 4, 2])


In [33]:
class Embedder(nn.Module):
  def __init__(self, ntoken, d_model):
    super(Embedder, self).__init__()
    #tokenの数と、分散表現の次元数
    self.embeddings = nn.Embedding(ntoken, d_model)
  
  def forward(self, x):
    x = self.embeddings(x)
    return x


class LSTM_Layer(nn.Module):
  def __init__(self, d_model, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(d_model, hidden_size, batch_first=True)

  #入力と(h, c)のタプル
  def forward(self, x, hidden):
    output, hidden = self.lstm(x, hidden)
    return output, hidden

  
  
class ClassificationHead(nn.Module):
  def __init__(self, d_model, output_dim):
    super().__init__()
    self.linear = nn.Linear(d_model, output_dim)
    nn.init.normal_(self.linear.weight, std=0.02)
    nn.init.normal_(self.linear.bias, 0)

  def forward(self, x):
    output = self.linear(x)
    return output

class LSTM_Classification(nn.Module):
  def __init__(self, batch_size, ntoken, d_model, hidden_size, output_dim):
    super().__init__()
    self.batch_size = batch_size
    self.hidden_size = hidden_size
    self.net1 = Embedder(ntoken, d_model)
    self.net2 = LSTM_Layer(d_model, hidden_size)
    self.net3 = ClassificationHead(hidden_size, output_dim)

  def forward(self, x, hidden):
    x1 = self.net1(x) # [batch_size, ntoken, d_model]
    x2, (ht, ct) = self.net2(x1, hidden) # [batch_size, ntoken, hidden_size], ([1, batch_size, hidden_size], [1, batch_size, hidden_size])
    #隠れ状態の最後を使う
    out = self.net3(ht[-1]) 
    return out, hidden

  def init_hidden(self, device):
    return (torch.zeros(1*1, self.batch_size, self.hidden_size, device=device),
            torch.zeros(1*1, self.batch_size, self.hidden_size, device=device))

In [34]:
#テスト
batch = next(iter(train_dl))
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

# モデル構築
net = LSTM_Classification(64, len(TEXT.vocab.stoi), 300, 256, 5)
hidden = (torch.zeros(1*1, 64, 256),
            torch.zeros(1*1, 64, 256))

#hidden = net.init_hidden(device)

# 入出力
x = batch.text[0]
x1, hidden = net(x, hidden)

print("入力のテンソルサイズ：", x.shape)
print("出力のテンソルサイズ：", x1.shape)
print(hidden[0].shape)

入力のテンソルサイズ： torch.Size([64, 37])
出力のテンソルサイズ： torch.Size([64, 5])
torch.Size([1, 64, 256])
